In [1]:
from google.colab import drive
drive.mount('/content/drive')
# 1) 升级 pip
!pip install --upgrade pip

# 2) 安装 PyTorch（默认用 CUDA，如果你想用 CPU 版可去掉 --index-url 一行）
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu113

# 3) 安装其它常用库
!pip install numpy pandas pymatgen scikit-learn h5py tqdm

# 4) Clone 你的 DFT 项目（请将下面的 URL 换成你自己的仓库地址，如果不是公开仓库，可以先把代码上传到 Colab 再做 clone）
import os
os.chdir('/content')
if not os.path.isdir('DFT'):
    # 假设你的项目在 GitHub 上，替换成你自己的仓库地址
    !git clone https://github.com/ChenHongBo0420/DFT.git

# 5) 进入项目根目录并用 pip editable 模式安装
os.chdir('/content/DFT')
print("当前目录：", os.getcwd())
print("目录列表：", os.listdir(os.getcwd()))

!pip install -e .
# 定义 CSV 的绝对路径
TRAIN_CSV = "/content/drive/MyDrive/DFT_CSVs/Train.csv"
VAL_CSV   = "/content/drive/MyDrive/DFT_CSVs/Val.csv"
PRED_CSV  = "/content/drive/MyDrive/DFT_CSVs/predict.csv"

# 先打印确认一下文件是否存在
import os
for p in (TRAIN_CSV, VAL_CSV, PRED_CSV):
    print(p, "→", "存在" if os.path.isfile(p) else "不存在")


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://download.pytorch.org/whl/cu113
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64" (from torch) (from versions: none)
ERROR: No matching distribution found for nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64"
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 生成新的 MaxAbsScaler 并保存到 dftpy/scalers/
# 注意：请先打开 dftpy/scripts/generate_scalers.py，确认里面 train_list_csv = TRAIN_CSV（或你自己的路径）已经设置正确。
!python3 dftpy/scripts/generate_scalers.py

>>> 共计 11 个训练样本，max_atoms = 256，padding_size = 256
X_1.shape = (11, 256, 360)
X_2.shape = (11, 256, 360)
X_3.shape = (11, 256, 360)
X_4.shape = (11, 256, 360)
>>> 开始 fit MaxAbsScaler（C 类）: flat_C.shape = (2816, 360)
    完成 C 类
>>> 开始 fit MaxAbsScaler（H 类）: flat_H.shape = (2816, 360)
    完成 H 类
>>> 开始 fit MaxAbsScaler（N 类）: flat_N.shape = (2816, 360)
    完成 N 类
>>> 开始 fit MaxAbsScaler（O 类）: flat_O.shape = (2816, 360)
    完成 O 类
>>> 四个 MaxAbsScaler 已保存到：
     /content/DFT/dftpy/dftpy/scalers/Scale_model_C.joblib
     /content/DFT/dftpy/dftpy/scalers/Scale_model_H.joblib
     /content/DFT/dftpy/dftpy/scalers/Scale_model_N.joblib
     /content/DFT/dftpy/dftpy/scalers/Scale_model_O.joblib


In [4]:
# ──────────────────────────────────────────────────────────────
# 1. 确认 scalers 目录的位置和内容
# ──────────────────────────────────────────────────────────────
# 先看看实际的 scalers 文件夹在什么位置，以及里面有没有那四个 .joblib
!ls /content/DFT/dftpy/dftpy/scalers

# 如果这一行输出中能看到以下 4 个文件，就说明目录没问题：
#   Scale_model_C.joblib
#   Scale_model_H.joblib
#   Scale_model_N.joblib
#   Scale_model_O.joblib

# ──────────────────────────────────────────────────────────────
# 2. 把整个 scalers 目录打包为 scalers.zip
# ──────────────────────────────────────────────────────────────
# 注意：这里的路径必须和上一步“ls”看到的一致
!zip -r /content/scalers.zip /content/DFT/dftpy/dftpy/scalers

# ──────────────────────────────────────────────────────────────
# 3. 调用 Colab 的下载接口，把 scalers.zip 下载到本地
# ──────────────────────────────────────────────────────────────
from google.colab import files
files.download('/content/scalers.zip')


Scale_model_C.joblib  Scale_model_N.joblib
Scale_model_H.joblib  Scale_model_O.joblib
  adding: content/DFT/dftpy/dftpy/scalers/ (stored 0%)
  adding: content/DFT/dftpy/dftpy/scalers/Scale_model_C.joblib (deflated 63%)
  adding: content/DFT/dftpy/dftpy/scalers/Scale_model_H.joblib (deflated 60%)
  adding: content/DFT/dftpy/dftpy/scalers/Scale_model_N.joblib (deflated 59%)
  adding: content/DFT/dftpy/dftpy/scalers/Scale_model_O.joblib (deflated 58%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>